In [2]:
import pandas as pd
import pybaseball as pb
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Introduction

The motivation for this project is entirely due to me being sick of losing at fantasy baseball. For years I routinely have the strongest regular season team, only to fizzle out in the playoffs. Since COVID began, I've been hit especially hard by players sitting out.

Hopefully a data-driven approach (and a league of managers who probably won't look much deeper than Yahoo's top-level stats) will help me find some sleepers and finally get ahead

#### Data
Utilizing the *pybaseball* library, I'll pull the data from the previous season and use the stats (combined with the scoring settisn from our league) to ensure that it lines up with the final results from the previous season. This is a simple step, though not trivial, because if I don't do this and it turns out I'm wrong, and Yahoo has different data than MLB, then this'll never work. I'll create a new field that has the calculated Fantasy Points from Yahoo.

In [12]:
# Pull pybaseball data (min 10 PAs) from 2021 season
batters = pd.DataFrame(pb.batting_stats(2021,qual=10))

# Calculate fantasy points based on stats, verify it matches Yahoo (it does)
batters["Points"] = (batters["R"]*2 +
                    batters["H"]*1 +
                    batters["2B"]*1 +
                    batters["3B"]*2 +
                    batters["HR"]*3 +
                    batters["RBI"]*2 +
                    batters["SB"]*1 +
                    batters["CS"]*-1 +
                    batters["BB"]*1 +
                    batters["IBB"]*1 +
                    batters["HBP"]*1 +
                    batters["SO"]*-0.5)

#### Correlations
Next I'll see which variables available from MLB correlate the most with these Fantasy Points. Because I know *exactly* which variables Yahoo uses to calculate our Fantasy Points, I expect that they'll be included here. Certainly they'll give me a problem with regression later, but we'll cross that bridge when we get to it. From these correlations, I've taken the top 30 positively correlated stats (as they all had very high *r* values) and the top 10 negatively correlated. They'll probably fall out in regression selection later if they're no good.

In [42]:
# Calculate the stats that most highly correlate with Fantasy Points, drop NaNs
corrs = batters.corr().Points
corrs.dropna(inplace=True)
corrs = corrs.sort_values(ascending=False)
top_pos = corrs.head(30)
bot_pos = corrs.tail(10)
variables = list(top_pos.head(30).index) + list(bot_pos.head(10).index)

# Some of these are mostly NaNs, so we'll drop those columns
to_remove = ["Points",
             "SB-X (pi)",
            "wSB/C (pi)",
            "KN% (pi)",
            "KN%",
            "XX% (pi)",
            "KN% (sc)",
            "CS% (pi)",
            "vSB (pi)",
            "SB% (pi)"]

reg_variables = []
for par in variables:
    if par not in to_remove:
        reg_variables.append(par)

As it turned out, several of the fields were nearly full of NaNs (some knuckleballer stats and others that only a handful of players had numbers for) and had to be manually removed. 30 variables remained entering regression.

#### Regression
As I know exactly how Fantasy Points are calculated, there really isn't much mystery here, as there will be no error term and $R^2_{adj} = 1$. I may drop some of the variables that are directly used to calculate the Fantasy Points, but first let's see what it looks like with them in. Then it will be repeatedly removing variables and checking fits. I prefer this to any automated form of regression selection for this process, as my baseball knowledge can (hopefully) have some input on what's worth keeping and what's worth rejecting.

In [32]:
# Time for regression
y = batters["Points"]
x = batters[reg_variables]
bat_model = sm.OLS(y,x)
bat_residuals = bat_model.fit()

print(bat_residuals.summary())

                                 OLS Regression Results                                
Dep. Variable:                 Points   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          2.025e+05
Date:                Thu, 16 Dec 2021   Prob (F-statistic):                        0.00
Time:                        23:03:25   Log-Likelihood:                         -1897.1
No. Observations:                 732   AIC:                                      3852.
Df Residuals:                     703   BIC:                                      3986.
Df Model:                          29                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

As expected, the model is perfectly fit. That's pretty useless, and we already know the exact contribution of the stats that Yahoo already assigns a value to, so let's drop them from the model so we may discover stats that have predictive value that aren't already measured

In [43]:
y = batters["Points"]

reg_variables0 = reg_variables
reg_variables0.remove("R")
reg_variables0.remove("H")
reg_variables0.remove("2B")
reg_variables0.remove("HR")
reg_variables0.remove("RBI")
reg_variables0.remove("BB")

x = batters[reg_variables0]
bat_model = sm.OLS(y,x)
bat_residuals = bat_model.fit()

print(bat_residuals.summary())

                                 OLS Regression Results                                
Dep. Variable:                 Points   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          1.387e+04
Date:                Thu, 16 Dec 2021   Prob (F-statistic):                        0.00
Time:                        23:05:26   Log-Likelihood:                         -2965.6
No. Observations:                 732   AIC:                                      5977.
Df Residuals:                     709   BIC:                                      6083.
Df Model:                          23                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

That's still an uncomfortably high $R^2_{adj}$. Let's start removing non-significant ($p\geq0.05$) variables from the model and see how it responds.

In [44]:
y = batters["Points"]

reg_variables1 = reg_variables0
reg_variables1.remove("HardHit")
reg_variables1.remove("PA")
reg_variables1.remove("Rep")
reg_variables1.remove("Pitches")
reg_variables1.remove("Strikes")
reg_variables1.remove("Balls")
reg_variables1.remove("LD")
reg_variables1.remove("G")
reg_variables1.remove("Lg")
reg_variables1.remove("RAR")
reg_variables1.remove("WAR")
reg_variables1.remove("IFH")
reg_variables1.remove("K%+")
reg_variables1.remove("CSW%")

x = batters[reg_variables1]
bat_model = sm.OLS(y,x)
bat_residuals = bat_model.fit()

print(bat_residuals.summary())

                                 OLS Regression Results                                
Dep. Variable:                 Points   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          2.988e+04
Date:                Thu, 16 Dec 2021   Prob (F-statistic):                        0.00
Time:                        23:11:04   Log-Likelihood:                         -2996.1
No. Observations:                 732   AIC:                                      6012.
Df Residuals:                     722   BIC:                                      6058.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------